In [1]:
from lxml import etree as ET
import shutil
from subprocess import check_output
import yaml
from pathlib import Path
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Polygon

input = "C:/Users/mhaibt/Nextcloud/Uruk/Uruk-BasisVR/UrukVR_allDGPScameras.gpkg"
template = "C:/Users/mhaibt/Nextcloud/Uruk/Uruk-BasisVR/templatev2.rcbox"
outputfolder = "C:/Users/mhaibt/Nextcloud/Uruk/Uruk-BasisVR/"

def write_rcbox(series, tree, root):
    for centre in root.iter('centre'):
        centre.text = str(series['x_centro']) + ' ' + str(series['y_centro']) + ' ' + str(series['z_centro'])
    for widthHeightDepth in root.iter('widthHeightDepth'):
        widthHeightDepth.text = str(series['width']) + ' ' + str(series['height']) + ' ' + str(series['depth'])
    tree.write(series['rcboxoutpath'])
def createTileFrame(geodf):
    geo = geodf.geometry
    #geo_clean = geo[~(geo.is_empty | geo.isna())]
    geo_union = geo.unary_union
    geo_envelope = geo_union.envelope
    return geo_envelope
def createBuffer(grid_df):
    grid_geom = grid_df.geometry
    grid_df['buffer'] = grid_geom.geometry.buffer(50)  
    return grid_df
def getCameralist(series, geodf):
    
    cameraswithinlist = geodf.geometry.within(series.geometry)
    cameraswithin = geodf.loc[cameraswithinlist]
    series['cameralist']=list(cameraswithin['FileName'])
    return series
def getNeighbourlist(series, grid_df):
    neighbourlist = grid_df.geometry.intersects(series.geometry)
    neighbour = grid_df.loc[neighbourlist]
    series['neighbours'] = list(neighbour['GridId'])
    return series
def createGrid(geodf_envelope, length, width):
    xmin,ymin,xmax,ymax = geodf_envelope.total_bounds
    cols = list(range(int(np.floor(xmin)), int(np.ceil(xmax)), width))
    rows = list(range(int(np.floor(ymin)), int(np.ceil(ymax)), length))
    #rows.reverse()
    grid_df = gpd.GeoDataFrame()
    for ix,x in enumerate(cols): 
        for iy,y in enumerate(rows):
            grid={}
            grid['GridId']= str(int(ix)) + '_' + str(int(iy))
            grid['geometry'] = Polygon([(x,y), (x+width, y), (x+width, y+length), (x, y+length)])
            grid_df = grid_df.append(grid, True)
    return grid_df

def create_rcboxoutpath(series, outputfolder):
    series['rcboxoutpath'] = os.path.join(outputfolder, 'tile3D_' + str(series['id']) + '.rcbox')
    return series

def load_xml(name):
    tree = ET.parse(name)
    root = tree.getroot()
    return tree, root

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
geodf = gpd.read_file(input, layer='UrukVR_allDGPScameras')
geo_envelope = createTileFrame(geodf)
df = {'Name': ['envelope1'], 'geometry': geo_envelope}
geodf_envelope = gpd.GeoDataFrame( df,geometry= 'geometry')
grid_df = createGrid(geodf_envelope, 250, 250)
grid_df = createBuffer(grid_df)
grid_df.set_geometry('buffer')
grid_df = grid_df.apply(getCameralist, geodf=geodf, axis=1)
grid_df = grid_df.apply(getNeighbourlist, grid_df=grid_df, axis=1)
print(grid_df['neighbours'])


In [ ]:
tree, root = load_xml(template)
df = df.apply(create_rcboxoutpath, outputfolder=outputfolder, axis=1)
df.apply(write_rcbox, tree=tree, root=root, axis=1)